# import extensions

In [1]:
import pandas as pd
import numpy as np

# load the file

In [ ]:
df = pd.read_csv("C:/Users/user/Downloads/AviationData.csv/AviationData.csv", encoding='cp1252')
df

# drop columns with missing values

In [ ]:
# 1. Drop columns with more than 50% missing values
missing_threshold = 0.5
missing_percentages = df.isnull().mean()
columns_to_drop = missing_percentages[missing_percentages > missing_threshold].index
df = df.drop(columns=columns_to_drop)

print("Columns after dropping >50% missing values:", df.columns.tolist())

In [ ]:
# Step 1: Inspect the data
print("Initial dataset shape:", df.shape)
print("Columns with missing values:\n", df.isnull().sum())

# drop column with irrevelant columns

In [ ]:
# Drop irrelevant columns
# Assuming some columns are irrelevant based on their lack of meaningful data
columns_to_drop = [
    'Airport.Name', 'Latitude', 'Longitude', 
    'Publication.Date','Air.carrier','FAR.Description'
]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
df

# handle missing values

In [ ]:
# Step 3: Handle missing values
# Fill missing numerical values with mean/median
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())
df

#  fill categorical columns 

In [ ]:
# Fill missing categorical values with the mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])
df

In [ ]:
print("Initial dataset shape:", df.shape)
print("Columns with missing values:\n", df.isnull().sum())

# fill numerical columns and unknowns

In [ ]:
# Location - Missing values replaced with 'Unknown Location'
df['Location'] = df['Location'].fillna('Unknown Location')

# Country - Missing values replaced with 'Unknown Country'
df['Country'] = df['Country'].fillna('United States')

# Airport.Code - Replace with 'Unknown Airport' since it's a specific identifier
df['Airport.Code'] = df['Airport.Code'].fillna('NONE')

# Injury.Severity - Replace with 'Unknown Severity' for missing cases
df['Injury.Severity'] = df['Injury.Severity'].fillna('Unknown Severity')

# Aircraft.Damage - Replace with 'Unknown Damage'
df['Aircraft.damage'] = df['Aircraft.damage'].fillna('Unknown Damage')

# Aircraft.Category - Replace with 'Unknown Category'
df['Aircraft.Category'] = df['Aircraft.Category'].fillna('Unknown Category')

# Registration.Number - Replace with 'Unknown Registration'
df['Registration.Number'] = df['Registration.Number'].fillna('Unknown Registration')

# Make and Model - Replace missing values with 'Unknown Make/Model'
df['Make'] = df['Make'].fillna('Unknown Make')
df['Model'] = df['Model'].fillna('Unknown Model')

# Amateur.Built - Replace missing values with the mode (e.g., 'No')
df['Amateur.Built'] = df['Amateur.Built'].fillna(df['Amateur.Built'].mode()[0])

# Engine.Type - Replace with 'Unknown Engine Type'
df['Engine.Type'] = df['Engine.Type'].fillna('Unknown Engine Type')

# Schedule - Replace with 'Unknown Schedule'
df['Schedule'] = df['Schedule'].fillna('Unknown Schedule')

# Purpose.Of.Flight - Replace with 'Unknown Purpose'
df['Purpose.of.flight'] = df['Purpose.of.flight'].fillna('Unknown Purpose')
# Weather.Condition - Replace with 'Unknown Weather Condition'
df['Weather.Condition'] = df['Weather.Condition'].fillna('Unknown Weather Condition')

# Broad.Phase.Of.Flight - Replace with 'Unknown Phase'
df['Broad.phase.of.flight'] = df['Broad.phase.of.flight'].fillna('Unknown Phase')

# Report.Status - Replace with 'Unknown Status'
df['Report.Status'] = df['Report.Status'].fillna('Unknown Status')
df

# rename columns for consistency

In [ ]:
# Step 4: Rename columns for consistency
df.columns = df.columns.str.replace('.', ' ').str.strip().str.title()
df

# remove duplicates

In [ ]:
# Step 5: Remove duplicate rows if any
df = df.drop_duplicates()
df

# create a severity index

In [ ]:
# Create a new 'Severity Index' to aggregate injury severity
df['Severity Index'] = df['Total Fatal Injuries'] * 3 + df['Total Serious Injuries'] * 2 + df['Total Minor Injuries']
df

# groupby the dates into one

In [ ]:
# Convert 'Event Date' to datetime format
df['Event Date'] = pd.to_datetime(df['Event Date'], errors='coerce')

# Extract the year from 'Event Date'
df['Year'] = df['Event Date'].dt.year

# Group by 'Year' and count the number of unique 'Accident Number'
accident_counts = df.groupby('Year')['Accident Number'].nunique().reset_index()
accident_counts.columns = ['Year', 'Yearly_Accident_Counts']

# Merge the accident counts back to the original DataFrame
df = df.merge(accident_counts, on='Year', how='left')

# Display the updated DataFrame
print(df.head())


In [ ]:

# Get the current year
current_year = pd.to_datetime('today').year

# Filter the data to the last 20 years
df_last_20_years = df[df['Year'] >= (current_year - 20)]

# Optional: Save the filtered data to a new CSV file
df_last_20_years.to_csv("filtered_last_20_years.csv", index=False)

# Display the filtered data
print(df_last_20_years.head())

In [ ]:
df

# create a new column to check risk levels

In [ ]:
# Injury Severity Index
df['Total Injuries'] = (
    df['Total Fatal Injuries'] + 
    df['Total Serious Injuries'] + 
    df['Total Minor Injuries']
)
df['Risk Score'] = df['Total Injuries'] / (df['Total Injuries'] + df['Total Uninjured'] + 1)

# Engineer Year column if not already present
if 'Year' not in df.columns:
    df['Year'] = pd.to_datetime(df['Event Date']).dt.year

# Map injury severity to risk levels
severity_mapping = {
    'Fatal': 3,
    'Serious': 2,
    'Minor': 1,
    'None': 0
}
df['Severity Level'] = df['Injury Severity'].map(severity_mapping)

print(df[['Risk Score', 'Severity Level']].head())



# export the cleaned data

In [ ]:
# Step 6: Export the cleaned dataset
new_file_path = 'C:/Users/user/Downloads/AviationData_clean.csv'
df.to_csv(new_file_path, index=False)

In [ ]:
print("Cleaned dataset shape:", df.shape)
print(f"Cleaned data saved to: {'C:/Users/user/Downloads/AviationData_new.csv'}")